# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [79]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [80]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

init_x0 (generic function with 1 method)

In [444]:


function Compute_Alpha_dual(v_k, dir_v_k)
    n = length(v_k)
    if n == 0
        return 0.
    end
    alphas = ones(n)
    for i=1:n
        if dir_v_k[i] <= 0
            alphas[i] = -v_k[i] * 0.95 / dir_v_k[i]
        end
    end
    return minimum(alphas)
end


    
function Compute_Alpha_primal(v_k, dir_v_k, lvar, uvar)
    n = length(v_k)
    alpha_l, alpha_u = ones(n), ones(n)
    for i=1:n
        if dir_v_k[i] > 0
            alpha_u[i] = (uvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        elseif dir_v_k[i] < 0
            alpha_l[i] = (lvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        end
    end
    return min(minimum(alpha_l), minimum(alpha_u))
end

function Compute_mu(x_l, x_u, s, stilde, lvar, uvar)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s' * (x_l-lvar) + stilde' * (uvar-x_u)) / (length(x_l) + length(x_u))
end

Compute_mu (generic function with 1 method)

In [529]:

function MehrotraPCQuadBounds(QM, max_iter; eps=1e-10, tol_step_x=1e-8, eps_mu=1e-8, display=true)
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    x_k = init_x0(lvar, uvar)
    id_non_inf_l, id_non_inf_u = findall((x -> x!=-Inf), lvar), findall((x -> x!=Inf), uvar) # finite bounds index
    @assert all(x_k .> lvar) && all(x_k .< uvar)
    A = jac(QM, x_k)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_k)
    c = grad(QM, zeros(n_cols))
    c0 = obj(QM, zeros(n_cols))
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    s_k_l, s_k_u = zeros(n_cols), zeros(n_cols)
    s_k_l[id_non_inf_l] .= 1.
    s_k_u[id_non_inf_u] .= 1.
    lambda_k = Matrix(A)'\(c -s_k_l+s_k_u+Q*x_k) # least square initialisation, s_0 = stilde_0
    
    rb_k = A * x_k - b
    rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
    mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # matrices without infinity constraints
    nb_non_inf_l, nb_non_inf_u = length(id_non_inf_l), length(id_non_inf_u) # number of finite constraints
    
    Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
    Sk_non_inf_l = spzeros(nb_non_inf_l, n_cols)
    I_non_inf_l = spzeros(n_cols, nb_non_inf_l)
    for j=1:nb_non_inf_l   # cartesian index not working when nb_non_inf_l = 0
        Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        I_non_inf_l[id_non_inf_l[j], j] = 1
    end
    
    Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
    Sk_non_inf_u = spzeros(nb_non_inf_u, n_cols)
    I_op_non_inf_u = spzeros(n_cols, nb_non_inf_u)
    for j=1:nb_non_inf_u 
        Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        I_op_non_inf_u[id_non_inf_u[j], j] = -1
    end
    Lvar_non_inf = Diagonal(lvar[id_non_inf_l]) 
    Uvar_non_inf = Diagonal(uvar[id_non_inf_u]) 

    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps # start false?
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    

    while k<max_iter && (!opti_pdd || !opti_rb || !opti_rc) && !small_step_x && !small_mu
 
        # Affine scaling direction
        Jacob_Fk = [-Q            A'                             I_non_inf_l                         I_op_non_inf_u
                    A             spzeros(n_rows, n_rows)        spzeros(n_rows, nb_non_inf_l)       spzeros(n_rows, nb_non_inf_u)
                    Sk_non_inf_l  spzeros(nb_non_inf_l, n_rows)  Xk_non_inf_l-Lvar_non_inf           spzeros(nb_non_inf_l, nb_non_inf_u)
                    Sk_non_inf_u  spzeros(nb_non_inf_u, n_rows)  spzeros(nb_non_inf_u,nb_non_inf_l)  Xk_non_inf_u-Uvar_non_inf]
        Fk_aff = [-rc_k
                  -rb_k
                  -(x_k[id_non_inf_l]-lvar[id_non_inf_l]).*s_k_l[id_non_inf_l]
                  -(x_k[id_non_inf_u]-uvar[id_non_inf_u]).*s_k_u[id_non_inf_u]]

        dir_aff_k = Jacob_Fk\Fk_aff
        
        alpha_aff_pri = Compute_Alpha_primal(x_k, dir_aff_k[1:n_cols], lvar, uvar)
        alpha_aff_dual_l = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                            dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        alpha_aff_dual_u = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                                dir_aff_k[n_rows+n_cols+nb_non_inf_l+1:end])
        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        if nb_non_inf_l == 0
            alpha_aff_dual_final = alpha_aff_dual_u
        elseif nb_non_inf_u == 0
            alpha_aff_dual_final = alpha_aff_dual_l
        else
            alpha_aff_dual_final = min(alpha_aff_dual_l, alpha_aff_dual_u)
        end
        
        mu_aff = Compute_mu(x_k[id_non_inf_l] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_l],
                    x_k[id_non_inf_u] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_u],
                    s_k_l[id_non_inf_l] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l],
                    s_k_u[id_non_inf_u] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end],
                    lvar[id_non_inf_l], uvar[id_non_inf_u])
        
        sigma = (mu_aff / mu_k)^3

        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e[1:nb_non_inf_l] - dir_aff_k[1:n_cols][id_non_inf_l].*dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
                 sigma*mu_k*e[1:nb_non_inf_u] - dir_aff_k[1:n_cols][id_non_inf_u].*dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end]] 
        
        dir_cc_k = Jacob_Fk\Fk_cc
        dir_k = dir_aff_k .+ dir_cc_k # final direction
        
        alpha_max_pri = Compute_Alpha_primal(x_k, dir_k[1:n_cols], lvar, uvar)
        alpha_l_max_dual = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                        dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        
        alpha_u_max_dual = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                            dir_k[n_rows+n_cols+nb_non_inf_l+1: end])
        
    
        # new parameters
        alpha_k_pri = min(0.99*alpha_max_pri, 1)
        alpha_k_dual_l = min(0.99*alpha_l_max_dual, 1)
        alpha_k_dual_u = min(0.99*alpha_u_max_dual, 1)
        if nb_non_inf_l == 0
            alpha_dual_final = alpha_k_dual_u
        elseif nb_non_inf_u == 0
            alpha_dual_final = alpha_k_dual_l
        else
            alpha_dual_final = min(alpha_k_dual_l, alpha_k_dual_u)
        end
        
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_dual_final * dir_k[n_cols+1: n_rows+n_cols]
        s_k_l[id_non_inf_l] += alpha_dual_final * dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
        s_k_u[id_non_inf_u] += alpha_dual_final * dir_k[n_rows+n_cols+nb_non_inf_l+1: end]
        
        Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
        for j=1:nb_non_inf_l
            Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        end
        Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
        
        for j=1:nb_non_inf_u
            Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        end
        
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        
        k += 1
        
        if display == true
            @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_l_opt" => s_k_l, "s_u_opt" => s_k_u, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [530]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [531]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [532]:
res_mpc1 =  MehrotraPCQuadBounds(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -4.00e+00    |         1.20e+00       |    5.59e-01  |    1.72e-01  |  0.00e+00  |  1.00e+00
   1 |      9.46e+00    |         1.02e-02       |    1.01e-01  |    1.77e-01  |  2.11e+00  |  2.87e+00
   2 |      1.29e+01    |         2.70e-02       |    1.09e-02  |    1.85e-02  |  3.48e-01  |  1.86e-01
   3 |      1.35e+01    |         2.14e-03       |    7.33e-04  |    1.24e-03  |  4.59e-02  |  1.13e-02
   4 |      1.35e+01    |         1.28e-04       |    4.37e-05  |    7.41e-05  |  3.23e-03  |  6.74e-04
   5 |      1.35e+01    |         7.66e-06       |    2.61e-06  |    4.42e-06  |  1.92e-04  |  4.01e-05
   6 |      1.35e+01    |         4.59e-07       |    1.56e-07  |    2.64e-07  |  1.15e-05  |  2.39e-06
   7 |      1.35e+01    |         2.75e-08       |    9.28e-09  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [8.19348e-10, 3.0, 1.16064e-9]
  "lambda_opt" => [-9.46637, 12.0]
  "s_l_opt"    => [1.46637, 7.88256e-9, 0.466373]
  "s_u_opt"    => [0.0, 0.0, 0.0]
  "n_iter"     => 8
  "pdd"        => 1.64299e-9
  "cond_rb"    => 5.53424e-10
  "cond_rc"    => 9.37971e-10

In [533]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
res_mpc_verif =  MehrotraPCQuadBounds(SM_verif, 20);
display_results(res_mpc_verif)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -4.00e+00    |         1.20e+00       |    5.59e-01  |    1.72e-01  |  0.00e+00  |  1.00e+00
   1 |      9.47e+00    |         1.07e-02       |    1.01e-01  |    1.77e-01  |  2.12e+00  |  2.87e+00
   2 |      1.29e+01    |         2.68e-02       |    1.09e-02  |    1.83e-02  |  3.48e-01  |  1.86e-01
   3 |      1.35e+01    |         2.15e-03       |    7.32e-04  |    1.24e-03  |  4.56e-02  |  1.13e-02
   4 |      1.35e+01    |         1.29e-04       |    4.37e-05  |    7.41e-05  |  3.23e-03  |  6.71e-04
   5 |      1.35e+01    |         7.75e-06       |    2.61e-06  |    4.43e-06  |  1.93e-04  |  4.00e-05
   6 |      1.35e+01    |         4.65e-07       |    1.56e-07  |    2.64e-07  |  1.15e-05  |  2.38e-06
   7 |      1.35e+01    |         2.79e-08       |    9.32e-09  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [-7.9316e-10, -3.0, -1.19962e-9]
  "lambda_opt" => [-9.43981, 12.0]
  "s_l_opt"    => [0.0, 0.0, 0.0]
  "s_u_opt"    => [1.43981, 7.86089e-9, 0.43981]
  "n_iter"     => 8
  "pdd"        => 1.67829e-9
  "cond_rb"    => 5.57e-10
  "cond_rc"    => 9.43971e-10

In [534]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
res_mpc_verif2 =  MehrotraPCQuadBounds(SM_verif2, 20);
display_results(res_mpc_verif2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.00e+00    |         8.33e-01       |    1.00e+00  |    4.08e-01  |  0.00e+00  |  1.00e+00
   1 |      1.20e-01    |         4.14e-01       |    1.00e-02  |    4.08e-03  |  1.17e+00  |  1.58e-01
   2 |      8.44e-03    |         2.86e-02       |    1.00e-04  |    3.23e-04  |  1.29e-01  |  9.69e-03
   3 |      5.02e-04    |         1.71e-03       |    5.92e-06  |    1.92e-05  |  9.64e-03  |  5.77e-04
   4 |      2.99e-05    |         1.02e-04       |    3.51e-07  |    1.14e-06  |  5.74e-04  |  3.43e-05
   5 |      1.78e-06    |         6.07e-06       |    2.08e-08  |    6.77e-08  |  3.41e-05  |  2.04e-06
   6 |      1.06e-07    |         3.61e-07       |    1.24e-09  |    4.02e-09  |  2.03e-06  |  1.22e-07
   7 |      6.29e-09    |         2.15e-08       |    7.35e-11  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [6.29346e-9, 0.5, 0.5]
  "lambda_opt" => [-1.52004e-8]
  "s_l_opt"    => [1.0, 1.53955e-8, 1.53955e-8]
  "s_u_opt"    => [0.0, 0.0, 0.0]
  "n_iter"     => 7
  "pdd"        => 2.14939e-8
  "cond_rb"    => 7.34763e-11
  "cond_rc"    => 2.38931e-10

# Lecture des donnees .SIF

In [535]:

function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0, name=name_pb)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

In [536]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes\\AFIRO.SIF"

In [537]:
qpdata2 = readqps(afiro)
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2, "QM2")
SM2 = SlackModel(QM2);
#displayQuadraticModel(SM2)

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [538]:
res_mpc2 =  MehrotraPCQuadBounds(SM2, 20, tol_step_x = 1e-10);
display_results(res_mpc2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      8.20e+00    |         1.13e+01       |    1.85e+01  |    7.29e-01  |  0.00e+00  |  1.00e+00
   1 |      1.16e+01    |         8.03e+00       |    1.85e+01  |    7.29e-01  |  3.95e+00  |  1.73e+00
   2 |      2.46e+01    |         3.70e+00       |    1.85e+01  |    7.29e-01  |  2.69e+00  |  1.22e+01
   3 |      5.04e+00    |         2.94e+00       |    1.85e+01  |    7.26e-01  |  4.91e+00  |  2.76e+03
   4 |      1.29e+01    |         1.32e+02       |    1.85e+01  |    7.25e-01  |  1.48e+00  |  5.20e+04
   5 |      1.63e+01    |         6.92e+01       |    1.85e+01  |    7.24e-01  |  1.30e+00  |  2.17e+06
   6 |      1.65e+01    |         6.88e+01       |    1.85e+01  |    7.24e-01  |  1.21e-01  |  2.23e+06
   7 |      1.65e+01    |         6.88e+01       |    1.85e+01  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.44868, 0.899108, 0.547545, 0.415722, 0.364856, 1.09304, 0.…
  "lambda_opt" => [-2.74528, 8580.15, -1.01426, 0.0707497, -22.8307, -129.848, …
  "s_l_opt"    => [9099.74, 2.48332, 0.708226, 107490.0, 337.886, 0.896924, 1.3…
  "s_u_opt"    => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  16.9247…
  "n_iter"     => 14
  "pdd"        => 68.7769
  "cond_rb"    => 18.4708
  "cond_rc"    => 0.724247

In [539]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3, "QM3")
SM3 = SlackModel(QM3);

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [540]:
res_mpc3 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
display_results(res_mpc3)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.18e-01  |  0.00e+00  |  6.18e+00
   1 |     -7.89e+02    |         9.99e-01       |    2.35e+03  |    2.68e-01  |  2.82e+01  |  6.75e+00
   2 |     -6.27e+02    |         9.98e-01       |    1.87e+03  |    1.93e-01  |  1.79e+01  |  5.76e+00
   3 |     -6.02e+02    |         9.98e-01       |    1.80e+03  |    1.91e-01  |  2.73e+00  |  5.69e+00
   4 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  1.47e-01  |  5.69e+00
   5 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  8.69e-03  |  5.68e+00
   6 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  5.17e-04  |  5.68e+00
   7 |     -6.01e+02    |         9.98e-01       |    1.79e+03  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.739753, 2.88353, 0.660354, 0.41812, 0.579643, 0.761097, 0.…
  "lambda_opt" => [0.0397513, -0.0482735, 0.927659, -1.88853, 11.7148, 1.39946,…
  "s_l_opt"    => [1.86562, 1.24032, 2.8032, 4.00948, 2.31449, 2.58981, 8.12619…
  "s_u_opt"    => [0.0, 0.553963, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.…
  "n_iter"     => 14
  "pdd"        => 0.998339
  "cond_rb"    => 1793.76
  "cond_rc"    => 0.190701

In [541]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4, "QM4")
SM4 = SlackModel(QM4);

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [542]:
res_mpc4 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
display_results(res_mpc4)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.18e-01  |  0.00e+00  |  6.18e+00
   1 |     -7.89e+02    |         9.99e-01       |    2.35e+03  |    2.68e-01  |  2.82e+01  |  6.75e+00
   2 |     -6.27e+02    |         9.98e-01       |    1.87e+03  |    1.93e-01  |  1.79e+01  |  5.76e+00
   3 |     -6.02e+02    |         9.98e-01       |    1.80e+03  |    1.91e-01  |  2.73e+00  |  5.69e+00
   4 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  1.47e-01  |  5.69e+00
   5 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  8.69e-03  |  5.68e+00
   6 |     -6.01e+02    |         9.98e-01       |    1.79e+03  |    1.91e-01  |  5.17e-04  |  5.68e+00
   7 |     -6.01e+02    |         9.98e-01       |    1.79e+03  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.739753, 2.88353, 0.660354, 0.41812, 0.579643, 0.761097, 0.…
  "lambda_opt" => [0.0397513, -0.0482735, 0.927659, -1.88853, 11.7148, 1.39946,…
  "s_l_opt"    => [1.86562, 1.24032, 2.8032, 4.00948, 2.31449, 2.58981, 8.12619…
  "s_u_opt"    => [0.0, 0.553963, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.…
  "n_iter"     => 14
  "pdd"        => 0.998339
  "cond_rb"    => 1793.76
  "cond_rc"    => 0.190701

In [543]:
T = typeof(SM3.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [544]:
typeof(SM3.model) == QuadraticModel

true